# IGRINS Slit Viewing Camera and TESS *Re-Oriented!*

The header info in the IGRINS Slit Viewing Camera was malformed.  Let's attempt to fix it so we can preserve our nice figures.

In [ ]:
import lightkurve as lk

In [ ]:
lk.__version__

In [ ]:
# %load /Users/obsidian/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
import os

In [ ]:
import aplpy

In [ ]:
aplpy.__version__

In [ ]:
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import (PercentileInterval, SqrtStretch,
                                   ImageNormalize)
from astropy.io import fits
from astropy.wcs import WCS
import montage_wrapper as montage

In [ ]:
from reproject import reproject_interp, reproject_exact

In [ ]:
pos = SkyCoord(ra='10 49 11.95', dec='-53 19 01.8', unit=(u.hourangle, u.deg), frame='icrs')

### What imaging do we want?

| source | band | filename |
| -- | -- | -- |
| IGRINS Slit Viewing Camera (nights 1,2,3, & 4?) | K | `data/...`|
| HST? | H | `data/...`|
| TESS | R | `data/...`|

In [ ]:
rad_value = 60
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
def make_postage_stamp(im_in, output, hdu=None, vmin=None, vmax=None, radius=None):
    '''Make a postage stamp image'''
    fig = plt.figure(figsize=(5,5))
    if radius is None:
        radius = 60 * u.arcsecond
    if hdu is None:
        mosaic = aplpy.FITSFigure(im_in, north=True, figure = fig)
    else:
        mosaic = aplpy.FITSFigure(im_in,hdu=hdu, north=True, figure = fig)
    mosaic.recenter(pos.ra,pos.dec, radius=radius.to(u.degree).value)  # degrees
    if (vmin is not None) & (vmax is not None):
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear',
                              vmin=vmin, vmax=vmax)
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear',
                              vmin=vmin, vmax=vmax)        
    else:
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear')
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear')
    mosaic.tick_labels.set_font(size='small')
    #mosaic.tick_labels.set_xformat("hh:mm:ss")
    mosaic.set_theme('publication')

    aplpy.AxisLabels(mosaic).hide()
    aplpy.TickLabels(mosaic).hide()

    mosaic.add_scalebar(10.0 * u.arcsecond)
    #fig.scalebar.show(1.0/3600.0*4.0)  # length in degrees
    #mosaic.scalebar.set_length(4.0 * u.arcsecond)
    mosaic.scalebar.set_label("$10^{\prime\prime}$")
    #mosaic.scalebar.set_corner('bottom right')
    mosaic.scalebar.set_color('red')
    mosaic.scalebar.set_linestyle('solid')
    mosaic.scalebar.set_linewidth(5)  # points
    mosaic.scalebar.set_font(size=12, weight='medium', \
                          stretch='normal', family='sans-serif', \
                          style='normal', variant='normal')

    mosaic.save(output, dpi=300)

### 2MASS $J$ 

In [ ]:
im_in = '../../data/imaging/2MASS/aK_asky_990517s0370033.fits'
output = 'Luhman16_{}x{}arcsec_2M_K.png'.format(diam, diam)

In [ ]:
%%time
make_postage_stamp(im_in, output, radius=rad)

### TESS FFI

In [ ]:
sr = lk.search_targetpixelfile('Luhman 16')

In [ ]:
tpf = sr[0].download()

In [ ]:
ax = tpf.plot()

In [ ]:
tpf.ccd, tpf.camera

In [ ]:
im_in = '../../data/imaging/TESS/tess2021089092906-s0036-3-1-0207-s_ffic.fits'
im_in = fits.open(im_in)
output = 'Luhman16_{}x{}arcsec_TESS_S36.png'.format(diam, diam)

In [ ]:
%%time
make_postage_stamp(im_in, output, hdu=1, vmin=150, vmax=400, radius=rad)

## HST

In [ ]:
fn = '../../data/HST/MAST/MAST_2021-06-29T1207/HST/hst_15884_01_wfc3_uvis_f814w_ie1m01/hst_15884_01_wfc3_uvis_f814w_ie1m01_drc.fits'
hdu = fits.open(fn)
im_in = hdu
output = 'Luhman16_60x60arcsec_HST.png'

In [ ]:
hdu.info()

In [ ]:
rad_value = 10.5
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
%%time
make_postage_stamp(im_in, output, hdu=1, vmin=-0.1, vmax=3, radius=rad)

The FITS header is malformed.

### IGRINS Slit Viewing Camera

In [ ]:
rad_value = 21
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
fn = '../../data/IGRINS/originals/GS-2021A-DD-104/20210311/SDCS_20210311_0002.fits'
hdu = fits.open(fn)

In [ ]:
im_in = hdu
output = 'Luhman16_{}x{}arcsec_IGRINS_20210311.png'.format(diam, diam)

In [ ]:
hdu[1].header['CROTA1'] += 90
hdu[1].header['CROTA2'] += 90

In [ ]:
%%time
make_postage_stamp(im_in, output, hdu=1, vmin=0, vmax=4000, radius=rad)

### Zoom in ...

In [ ]:
rad_value = 10.5
rad = rad_value*u.arcsecond
diam = rad_value*2

In [ ]:
im_in = hdu
output = 'Luhman16_{}x{}arcsec_IGRINS_20210311.png'.format(diam, diam)

In [ ]:
%%time
make_postage_stamp(im_in, output, hdu=1, vmin=0, vmax=4000, radius=rad)

## Add axis labels to make a stand-alone graphic 

In [ ]:
def make_postage_stamp_with_labels(im_in, output, hdu=None, vmin=None, vmax=None, radius=None):
    '''Make a postage stamp image'''
    fig = plt.figure(figsize=(5,5))
    if radius is None:
        radius = 60 * u.arcsecond
    if hdu is None:
        mosaic = aplpy.FITSFigure(im_in, north=True, figure = fig)
    else:
        mosaic = aplpy.FITSFigure(im_in,hdu=hdu, north=True, figure = fig)
    mosaic.recenter(pos.ra,pos.dec, radius=radius.to(u.degree).value)  # degrees
    if (vmin is not None) & (vmax is not None):
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear',
                              vmin=vmin, vmax=vmax)
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear',
                              vmin=vmin, vmax=vmax)        
    else:
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear')
        mosaic.show_grayscale(invert=True, interpolation='none', stretch='linear')
    mosaic.tick_labels.set_font(size=12)
    mosaic.tick_labels.set_xformat("hh:mm:ss")
    mosaic.set_theme('talk')

    #aplpy.AxisLabels(mosaic).hide()
    #aplpy.TickLabels(mosaic).hide()

    mosaic.add_scalebar(10.0 * u.arcsecond)
    #fig.scalebar.show(1.0/3600.0*4.0)  # length in degrees
    #mosaic.scalebar.set_length(4.0 * u.arcsecond)
    mosaic.scalebar.set_label("$10^{\prime\prime}$")
    #mosaic.scalebar.set_corner('bottom right')
    mosaic.scalebar.set_color('#d35400')
    mosaic.scalebar.set_linestyle('solid')
    mosaic.scalebar.set_linewidth(5)  # points
    mosaic.scalebar.set_font(size=12, weight='medium', \
                          stretch='normal', family='sans-serif', \
                          style='normal', variant='normal')

    mosaic.save(output, dpi=300)

In [ ]:
%%time
make_postage_stamp_with_labels(im_in, "Luhman16_IGRINS_slit_20210311.pdf", hdu=1, vmin=0, vmax=4000, radius=rad)